# All settings will be given to the channel 1 output, in the DC Amp out configuration

In [1]:
%matplotlib widget 

In [2]:
import numpy as np
import pandas as pd
import time
import matplotlib
import matplotlib.pyplot as plt
import pyvisa as visa
from matplotlib import animation
from time import sleep
from matplotlib.animation import FuncAnimation
#Importing the functions from the module and the pulses dictionaries
from Sweeping import *
from Dictionaries import *
from PulseFiles import *
from Instrument import *
import pyqtgraph as pg 
import nidaqmx

# Visa interface for the M8190A instrument:

In [3]:
M8190A = VisaR(AWG_Settings4,60000)

In [4]:
Initialization(M8190A,AWG_Settings4)

Instruments Sampling Frecuency set to 5.00000000000000E+08Hz
Instruments DC1 Output route Voltage set to 7.00000000000000E-01deciVolts
AWG set to TRIGGERED Mode
Trigger In threshold value set to 1.00000000000000E+00V


In [5]:
M8190A.query('*IDN?')

'Agilent Technologies,M8190A,MY57701776,5.6.0.0-2'

In [6]:
M8190A.write('*RST')

5

In [5]:
M8190A.query('FREQ:RAST?')

'5.00000000000000E+08'

In [7]:
M8190A.query('SYST:ERR?')

'0,"No error"'

In [4]:
print(M8190A)

TCPIPInstrument at TCPIP0::ibn3-036.ibn-net.kfa-juelich.de::hislip0::INSTR


# Loading and triggering a single waveform:

In [6]:
M8190A.query('ARM:TRIG:LEV?')

'1.00000000000000E+00'

In [6]:
M8190A.write('ABOR')

5

In [5]:
test_run = Sequence_Pulse_List(PList_secondsA,PList_secondsB,10,0,15,7500000,M8190A,AWG_Settings4,1)

Current Segment Catalogue is 1,7500000  [(segment id, Segment size)]
Current Segment Catalogue is 1,7500000,2,7500000  [(segment id, Segment size)]
Sequence advancement method is COND
Sequence loaded with the following segment data "1,1,0,1,0,7499999,2,1,0,1,0,7499999"


In [6]:
test_run[1]

,Y1,SyncMarker1,SampleMarker1
0,0.000000,1,1
1,0.714286,1,1
2,0.714286,1,1
3,0.714286,1,1
4,0.714286,1,1
...,...,...,...
7499995,0.000000,1,1
7499996,0.000000,1,1
7499997,0.000000,1,1
7499998,0.000000,1,1


In [7]:
test_run[1]['Y1']*AWG_Settings4['Voltage Amplitude']

0            0.0
1          500.0
2          500.0
3          500.0
4          500.0
           ...  
7499995      0.0
7499996      0.0
7499997      0.0
7499998      0.0
7499999      0.0
Name: Y1, Length: 7500000, dtype: float64

In [8]:
test_run[2]['Y1']*AWG_Settings4['Voltage Amplitude']

0            0.0
1          500.0
2          500.0
3          500.0
4          500.0
           ...  
7499995      0.0
7499996      0.0
7499997      0.0
7499998      0.0
7499999      0.0
Name: Y1, Length: 7500000, dtype: float64

In [9]:
fig,ax= plt.subplots(2)

ax[0].grid(True)
ax[1].grid(True)
fig.suptitle('Pulses Loaded into the AWG')


ax[0].plot(test_run[3],test_run[1]['Y1']*AWG_Settings4['Voltage Amplitude'],label= 'Pulse A',color='blue')
ax[1].plot(test_run[3],test_run[2]['Y1']*AWG_Settings4['Voltage Amplitude'],label= 'Pulse B',color='red')
ax[0].set_ylabel('V  [mV]')
ax[0].set_xlabel('t  [ms]')
ax[1].set_ylabel('V  [mV]')
ax[1].set_xlabel('t  [ms]')

#ax.set_title('Pulses Loaded into the AWG')

fig.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()
#fig.savefig('Pulse_sweep_t=20',optimize=True,bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
Trigger_Pulse('DAQBNC1','ao1',1.5,4)

Triggering Pulse Stoped


In [8]:
def DAQ_Measuringhalf(DAQ_settings,playingtime):
    """This function starts sets up the DAQ box in order to collect data for a time duration given by "playing time"
      It then uses the DAQ box to trigger the AWG into playing a waveform.

      playingtime should be in seconds.
      
    """
    #Calculating the number of samples given the samplig frecuency and playying time
    samples = DAQ_settings['Sampling Frequency'] * playingtime * 1e-3
    time = np.linspace(0,playingtime,int(samples))

    #setting the measuring task
    measuring_task = nidaqmx.Task()
    
    #voltage measuring channel
    measuring_task.ai_channels.add_ai_voltage_chan("{a}/{b}".format(a = DAQ_settings['DAQ Name'], b = DAQ_settings['Analog Channel Input']),min_val=DAQ_settings['Minimum Voltage'],max_val= DAQ_settings['Maximum Voltage'])

    #Sampling configuration
    measuring_task.timing.cfg_samp_clk_timing(DAQ_settings['Sampling Frequency'], source="", active_edge=Edge.RISING, sample_mode=AcquisitionType.FINITE, samps_per_chan=int(samples))

    measuring_task.start()

    data = np.array(measuring_task.read(int(samples)))

    measuring_task.stop()
    measuring_task.close()

    return data,time

In [12]:
25*1e-3

0.025

In [22]:
DAQ_Settings1['Sampling Frequency'] * 25 *1e-3

10000.0

In [6]:
Trigger_Pulse('DAQBNC1','ao1',1.5,4)

Triggering Pulse Stoped


In [9]:
dataM, timeM = DAQ_Measuringhalf(DAQ_Settings1,25)

fig,ax= plt.subplots()

ax.grid(True)
#plt.plot(tom,data_off,label= 'AWG off')
plt.plot(timeM,dataM*1000,label= '{b} samples'.format(b=len(dataM)),color='orange')

ax.set_ylabel('V  [mV]')
ax.set_xlabel('t  [ms]')
ax.set_title('Voltage Amplitude measured with DAQ')
#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()
#fig.savefig('DAQ measurement',bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
M8190A.write('INIT:IMM')

9

In [4]:
dar,tar = DAQ_Measuring(DAQ_Settings1,25,1.5,M8190A)

fig,ax= plt.subplots()

ax.grid(True)
#plt.plot(tom,data_off,label= 'AWG off')
plt.plot(tar,dar*1000,label= '{b} samples'.format(b=len(dar)),color='orange')

ax.set_ylabel('V  [mV]')
ax.set_xlabel('t  [ms]')
ax.set_title('Voltage Amplitude measured with DAQ')
#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()
#fig.savefig('DAQ measurement',bbox_inches='tight')

Triggering Pulse Stoped


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
10/(2**14)

0.0006103515625

In [17]:
M8190A.write('ABOR')

5

There is something weird while initiating the function before setting up the DAQ

In [21]:
from nidaqmx.constants import Edge
from nidaqmx.constants import AcquisitionType


n = 10000

task = nidaqmx.Task()

task.ai_channels.add_ai_voltage_chan("DAQBNC1/ai0",min_val=-4,max_val=4)
task.timing.cfg_samp_clk_timing(400000, source="", active_edge=Edge.RISING, sample_mode=AcquisitionType.FINITE, samps_per_chan=n)


task.start()



data_on = np.array(task.read(n))



task.stop()
task.close


tom = np.linspace(0,25,n)


fig,ax= plt.subplots()

ax.grid(True)
#plt.plot(tom,data_off,label= 'AWG off')
plt.plot(tom,data_on*1000,label= '{b} samples'.format(b=n),color='orange')

ax.set_ylabel('V  [mV]')
ax.set_xlabel('t  [ms]')
ax.set_title('Voltage Amplitude measured with DAQ')
#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()
#fig.savefig('DAQ measurement',bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …